In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
pip show transformers accelerate


Name: transformers
Version: 4.47.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/ajain30/.local/lib/python3.9/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
---
Name: accelerate
Version: 1.1.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /home/ajain30/.local/lib/python3.9/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!pip install evaluate
!pip install metrics


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu117
  Using cached https://download.pytorch.org/whl/cu117/torch-2.0.1%2Bcu117-cp39-cp39-linux_x86_64.whl (1843.9 MB)
  Using cached https://download.pytorch.org/whl/triton-2.0.0-1-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new

In [4]:
pip install --upgrade transformers datasets evaluate torch


Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.5.1-cp39-cp39-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached torch-2.5.1-cp39-cp39-manylinux1_x86_64.whl (906.5 MB)
Using cached triton-3.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu117
    Uninstalling torch-2.0.1+cu117:
      Successfully uninstalled torch-2.0.1+cu117
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolve

In [8]:
import json
import os
import metrics
import evaluate
import random
from typing import List
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

# -------------------------
# Configuration
# -------------------------
MODEL_NAME = "bert-base-uncased"
MAX_LENGTH = 128
EPOCHS = 3
BATCH_SIZE = 8
OUTPUT_DIR = "./trained_model"  # directory to save the trained model and tokenizer

# -------------------------
# Load Data
# -------------------------
with open("ir_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# data expected keys: title, revision_id, summary, description, url, topic
descriptions = data["description"]
topics = data["topic"]

assert len(descriptions) == len(topics), "Descriptions and topics must have the same length."

# -------------------------
# Prepare Labels
# -------------------------
# Get unique topics
unique_topics = list(set(topics))
unique_topics.sort()  # ensure consistent ordering
label2id = {t: i for i, t in enumerate(unique_topics)}
id2label = {i: t for t, i in label2id.items()}

labels = [label2id[t] for t in topics]

# -------------------------
# Train-Test Split
# -------------------------
train_texts, val_texts, train_labels, val_labels = train_test_split(
    descriptions, labels, test_size=0.2, random_state=42, stratify=labels
)

# -------------------------
# Create Hugging Face Datasets
# -------------------------
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# -------------------------
# Tokenizer
# -------------------------
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Rename label column
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# -------------------------
# Model
# -------------------------
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(unique_topics),
    id2label=id2label,
    label2id=label2id
)

# -------------------------
# Training Arguments
# -------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    seed=42,
)

# -------------------------
# Metrics
# -------------------------
metric = evaluate.load("accuracy")  # Use `evaluate` library instead of `datasets`

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# -------------------------
# Trainer
# -------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# -------------------------
# Train
# -------------------------
trainer.train()

# -------------------------
# Save Model
# -------------------------
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Model training complete! Model and tokenizer are saved at:", OUTPUT_DIR)


Map:   0%|          | 0/40008 [00:00<?, ? examples/s]

Map:   0%|          | 0/10002 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/tmp/ipykernel_1245173/390948271.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.137200,1.171549,0.623775
2,0.989600,1.096863,0.661468
3,0.514800,1.230446,0.666667


Model training complete! Model and tokenizer are saved at: ./trained_model
